# Aggregations, Sorting, and Basic Calculations
© Explore Data Science Academy

## Learning Objectives

In this train we will explore:
- Built-in functions
- Grouping data
- Sorting data

## Outline

- Built-in functions
    - Counting
    - Basic Arithmetic
    - Sum and AVG
    - max and min
- ORDER BY
- GROUP BY
- HAVING

### Loading the database
Load SQL magic commands

In [ ]:
%load_ext sql

Load Chinook SQLite database

In [ ]:
%%sql 

sqlite:///chinook.db

Chinook database ER diagram:

<img src="https://github.com/Explore-AI/Pictures/blob/master/sqlite-sample-database-color.jpg?raw=true" width=70%/>

_[Image source](https://www.sqlitetutorial.net/sqlite-sample-database/)_

## [Built-in functions](https://db.apache.org/derby/docs/10.13/ref/rrefsqlj29026.html)

In this section, we demonstrate some of the useful SQL built-in functions for data analysis.

### 1. Counting
One of the most common built-in functions you'll encounter is the `count()` function. As the name implies, this function will count the number of rows of the column specified in its arguments. In the simplest case, it can be used as follows:

    SELECT count(column_name)
    FROM table_name

where the column whose rows we want to count is supplied as an argument into the count function.

Great, let's do some examples:

### 1.1. Counting rows of a given column

Let's write a query that counts the number of albums and the number of artists in the albums table. In English:

    return a count of the AlbumId column and a count of the ArtistId column from the albums table

In SQL:

In [ ]:
%%sql

SELECT count(AlbumId) AS "Number of Albums", count(ArtistId) AS "Number of Artists"
FROM albums;

Notice that we've added aliases for the counts for better readability. Try removing them and see what happens.

The above query seems to be successful right? However, those with a keen eye may be wondering why we have the same number of artists as albums, surely the number of albums should be greater than the number of artists. We'll discuss how to resolve this in the next example, in the meantime, do a `SELECT *` query on the albums table to see if you can understand where the problem is. 

In [ ]:
%%sql

-- your code here

### 1.2. Counting unique values in a given column
If you take a closer look at the albums table (i.e. in your SELECT query above), you will notice that some of the ArtistIds have been duplicated. This makes sense because some Artists will have written multiple albums. Now the question is, how do we reflect this in SQL?

This brings us to the `DISTINCT` keyword, which eliminates duplicates in a column. We can use this keyword together with the count function to only consider distinct values. 

Following from the previous example, write a query that counts the number of albums and the number of unique artists in the albums table. In English:

    return a count of the AlbumId column and a count of the unique rows in the ArtistId column from the albums table

In SQL:

In [ ]:
%%sql

SELECT count(AlbumId) AS "Number of Albums", count(DISTINCT ArtistId) AS "Number of Unique Artists"
FROM albums;

This looks more realistic, to achieve this we used the `DISTINCT` keyword in the argument of the count function. In other words, we supplied the count function with a new column (i.e. `DISTINCT ArtistId`) which only contains unique ArtistIds.

### 2. Max and Min

SQL has built-in functions for calculating the maximum value of column (i.e. `max()`) and the minimum value of column (i.e. `min()`). These functions can be applied on all numerical type columns and DATETIME columns. 

Let's do some examples.
### 2.1. Calculating the minimum of a column
Write a query that returns the name of the shortest song in the database. In English:

    return data from the Name column and the minimum of the Milliseconds column from the tracks table 
    
In SQL:

In [ ]:
%%sql

SELECT Name, min(Milliseconds) AS "Length in Millisecs"
FROM tracks;

Easy peasy, lemon squeezy. However, aside from the fact that this song is in Portuguese, nothing else makes sense. For example, how long is this song in minutes? 

Before we answer this question, let's do another example.

### 2.2. Calculating the maximum of a column
Write a query that returns the name of the most recently hired employee. In English:

    return the FirstName column,LastName column and maximum of the HireDate column from the employees table

In SQL:

In [ ]:
%%sql

SELECT FirstName, LastName, max(Hiredate) AS "Hire Date"
FROM employees;

As you can see, we can find the max (later) or min (earlier) dates in the same way as we do with numerical columns.

### 3. Basic arithmetic

We can perform basic calculations in SQL using standard math operators:
- `+` - addition
- `-` - subtraction
- `*` - multiplication
- `/` - division

Let's see some examples (since all these operators work in the same way, we will only demonstrate multiplication and division here):


### 3.1. Multiplication
Suppose chinook hired a South African accountant and needed to send them all company invoices. As such, each invoice total has to be converted from dollars to rands. Write a query that will achieve this (assumming that 1 Canadian dollar = 13.18 rands). In English:

    return data in all columns, the Total column multiplied by 13.18 from the invoices table

In SQL:

In [ ]:
%%sql

SELECT *, Total*13.81 AS "Total in Rands"
FROM invoices
LIMIT 10;

### 3.2. Division
To follow up on the query in subsection 2.1. Write a query that returns the name of the longest song in the database but converts the song length from Milliseconds to Minutes. In English:

    return data from the Name column and the maximum of the Milliseconds column divided by 60000 from the tracks table 

In SQL:   

In [ ]:
%%sql

SELECT Name, max(Milliseconds)/60000 AS "Length in Minutes"
FROM tracks;

Although our query does its job, we have stumbled upon a suspicious entry. An 88 minute song? 
Not exactly, before we jump to conclusions, notice that there is a MediaTypeId in the tracks table. If we trace that back to the media_types table:

In [ ]:
%%sql

SELECT *
FROM media_types

We can see that the database (particularly the tracks table) also stores videos :) so false alarm. That said, the `max()` and `min()` functions are also an ok way to find outliers in the data.

### 4. Summing and Averaging
Next to `max` and `min`, the `sum` and `avg` functions are the most common aggregator functions you will need in your SQL queries. It is worth noting that all these aggregator functions will ignore NULL values (i.e. missing entries in a column).

Let's do some examples.

### 4.1 Calculating the sum of a column
Write a query that calculates how long (in hours) it would take to go through all chinook media (i.e. music, videos, etc). In English:

    return the sum of the Milliseconds column divided by 3600000 from the tracks table

In SQL:

In [ ]:
%%sql

SELECT sum(Milliseconds)/3600000 AS "Time in Hours"
FROM tracks;

### 4.2 Calculating the average of a column

Write a query that returns the average purchase total per invoice. In English:

    return the average of the Total column from the invoices table

In SQL:

In [ ]:
%%sql

SELECT round(avg(Total),2) AS "Average purchase in dollars"
FROM invoices;

Note that we have also used the `round` in this query to round of our answer to 2 decimal places.

## Grouping and Ordering data

In this section we cover how to group and order data, we also explore how these operations can be combined with aggregator functions to produce more insightful queries.

### 1.  The ORDER BY clause
The ORDER BY clause is used to sort SQL query results by column a given column. This clause can be applied to a standard query as follows:

```sql
SELECT columns
FROM tables
WHERE boolean expression
ORDER BY column DESC
```

We can also specify whether we want the rows to be sorted in ascending order (default) or descending order (`DESC`) using `DESC` keyword. 

Let's do some examples


### 1.1. Sorting in ascending order
Write a query that returns the top 10 shortest media (in seconds) items in the database. In English:

    return the Name, Milliseconds column divided by 1000 from the tracks table, sort by the Milliseconds column in ascending order, and only output the first 10 rows
    
In SQL:

In [ ]:
%%sql

SELECT Name, round(Milliseconds/1000.0,2) AS "Length in Seconds"
FROM tracks
ORDER BY Milliseconds
LIMIT 10;

In this query, we divided by 1000.0 (i.e. adding the `.0`) to avoid integer division (which ignores the fractional parts). 

### 1.2. Sorting in descending order
Write a query that returns the Names of top 10 best customers and how much they spend on chinook media. In English:

    return the first and last name from the customers table, the Total column from the invoices table, align both tables by the customerId and sort the rows by the invoice Total column in descending order, and limit results to 10 rows

In SQL:

In [ ]:
%%sql

SELECT c.FirstName, c.LastName, i.Total AS "Purchase Total"
FROM customers c, invoices i
WHERE c.CustomerId = i.CustomerId
ORDER BY i.Total DESC
LIMIT 10;

### 2. The GROUP BY clause

The GROUP BY clause is used to group rows together which share the same value. In SQL we typically group rows by columns, which creates a group for each unique value in that column(s). As such, we typically use the GROUP BY clause with an aggregator function which will then operate on each group in the result set. A standard GROUP BY clause will take on the following format:

```
SELECT columns
FROM tables
WHERE boolean expression
GROUP BY columns 
```

The GROUP BY clause should always follow a WHERE clause (if it is present) and precede an ORDER BY clause (if it is present).

Let's do some examples:

### 2.1. Grouping by a single column

To start off, let's revisit the query from section 1.2 above. We tried to sort customers by their purchase totals. However, we forgot one crucial thing, customers generally make more than one purchase. As such, the best customers will be the ones with the highest sum of individual invoice totals.

Let's massage our query to reflect this, again write a query that returns the Names of top 10 best customers and how much they spend on chinook media. In English:

    return the first and last name from the customers table, the summed Total column from the invoices table, align both tables by the customerId and sort the rows by the invoice Total column in descending order, and group results by the CustomerId from the invoices table, and limit results to 10 rows
In SQL:

In [ ]:
%%sql

SELECT c.FirstName, c.LastName, round(sum(i.Total),2) AS "Total Spend"
FROM customers c, invoices i
WHERE c.CustomerId = i.CustomerId
GROUP BY i.CustomerId
ORDER BY [Total Spend] DESC
LIMIT 10; -- Remove this line to see the full output 

Now note that some of the customers that we initially thought were great customers (e.g. `Astrid Gruber`) are no longer there. 

### 2.2. Grouping by multiple columns
Write a query that returns the average unit price of media items in each genre. In English:

    return the genre Name column and an average of the unitprice column from the genre and tracks tables, where the genreId in the genre table matches the genreId in the tracks table, and the trackId in the tracks table matches the trackId in the invoice_items table, then group the results first by the track's genreId, and then group the results by the invoice_items unitprice
    
In SQL:

In [ ]:
%%sql

SELECT g.Name, round(avg(i.UnitPrice),2) AS "AVG Price per unit"
FROM genres g, tracks t, invoice_items i
WHERE g.GenreId = t.GenreId
    AND t.TrackId = i.TrackId
GROUP BY t.GenreId, i.UnitPrice
LIMIT 10; -- Remove this line to see the full output 

In this query, the information we need is spread across 3 tables. As such, we have used the WHERE clause to connect and align the information between them. When the GROUP BY clause is supplied with multiple columns with, it will group by each column consecutively starting with the leftmost column to the right. 

### 3. The HAVING Clause

Until now, we've been using the WHERE clause to return only the rows that meet a set of conditions. However, what we haven't been able to do is to specify conditions on aggregated rows (particularly rows that result from a GROUP BY clause), this is where the HAVING clause comes in. It works exactly like the WHERE clause but will operate on aggregated rows. As such, the HAVING clause always has to follow the GROUP BY clause, not the other way round. A standard HAVING clause will take on the following format:

```
SELECT columns
FROM tables
WHERE boolean expression
GROUP BY columns 
HAVING boolean expression
```

Let's see some examples:

### 3.1. Filtering the results of the group by clause

Write a query that returns the countries with 5 or more customers. In English:

    return a count of the CustomerId column, the country column from the customers table, group the results by the country name, and display only the countries with a count greater than or equal to 5
    
In SQL:

In [ ]:
%%sql 

SELECT count(CustomerId), Country
FROM customers
GROUP BY Country
HAVING COUNT(CustomerId) >= 5;

## Conclusion

In this train we have learned how to perform basic calculations within our SQL queries, how to sort query results in ascending or descending order, how to group data by the unique values of one or more columns, and lastly, how to place conditions on grouped rows of data.

## Additional Links 

- [Built-in Functions](https://db.apache.org/derby/docs/10.13/ref/rrefsqlj29026.html) 
- [The GROUP BY clause](https://db.apache.org/derby/docs/10.13/ref/rrefsqlj32654.html)
- [The ORDER BY clause](https://db.apache.org/derby/docs/10.13/ref/rrefsqlj13658.html)
- [The HAVING clause](https://db.apache.org/derby/docs/10.13/ref/rrefsqlj14854.html)